In [2]:
# packages
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

from sklearn.feature_extraction.text import TfidfVectorizer

# cut this down 
from nltk.stem.snowball import SnowballStemmer
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

import nltk
%matplotlib inline

In [3]:
# input from the webpage db
dbname = 'urx'
username = 'noahburbank'

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))

In [4]:
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM webpage_data_table;
"""
webpage_data = pd.read_sql_query(sql_query,con)
webpage_data.columns

Index([u'index', u'starturl', u'start_domain', u'start_titles',
       u'start_description', u'start_extracted_content',
       u'start_stemmed_word_list', u'_unit_it', u'endurl', u'end_domain',
       u'end_titles', u'end_decription', u'end_extracted_content',
       u'end_stemmed_word_list'],
      dtype='object')

In [8]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [43]:
from gensim import corpora, models, similarities

In [47]:
# function to get lists back into lists(they're actually sets though, so the order is an issue)
def strings_back_to_lists(string):
    string = string.replace('{', '')
    string = string.replace('}', '')
    mylist = string.split(',')
    return mylist

In [45]:
start_words = []
end_words = []

indexer = range(0,len(webpage_data))
for index in indexer:
    start_list = strings_back_to_lists(webpage_data['start_stemmed_word_list'][index])
    end_list = strings_back_to_lists(webpage_data['end_stemmed_word_list'][index])
    start_words.append(start_list)
    end_words.append(end_list)

In [ ]:
texts = start_words + end_words
dictionary = corpora.Dictionary(texts)
#print(dictionary.token2id)

In [51]:
dictionary.doc2bow(strings_back_to_lists(webpage_data['start_stemmed_word_list'][0]))

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1)]

In [53]:
import word2vec

In [64]:
sentences = webpage_data['start_extracted_content'] + webpage_data['end_extracted_content']
sentences = sentences + webpage_data['start_titles'] + webpage_data['end_titles']
sentences = sentences + webpage_data['start_description'] + webpage_data['end_decription']

model = models.Word2Vec(sentences, size=100, window = 5, min_count = 5, workers=4)


In [65]:
print model

Word2Vec(vocab=118, size=100, alpha=0.025)
